In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")


# Conéctese a OpenAI, Anthropic y Google; comente las líneas de Claude o Google si no las está usando

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()


def stream_gpt(prompt, system_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_claude(prompt, system_message):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

def stream_gemini(prompt, system_message):
    result = google.generativeai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message
    )
    messages=[
            {"role": "user", "parts": prompt}]
    response = ""
    stream = result.generate_content(messages, stream=True)
    for chunk in stream:
        response += chunk.text or ""
        yield response

# Una clase para representar una página web

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No se ha encontrado título de la página"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"



def stream_brochure(company_name, url, tone, model):
    prompt = f"Genera un folleto de la empresa {company_name}. Esta es su página de destino:\n"    
    prompt += Website(url).get_contents()

    system_message = "Eres un asistente que analiza el contenido de la página de inicio \
del sitio web de una empresa y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados."

    if tone == "Divertido":
        system_message += " Responde en formato Markdown de una manera divertida usando emojies."
    elif tone == "Jovial":
        system_message += " Responde en formato Markdown de una manera jovial usando emojies."
    elif tone == "Serio":
        system_message += " Responde en formato Markdown de una manera seria."
    elif tone == "Curioso":
        system_message += " Responde en formato Markdown de una manera curiosa usando emojies."

    if model=="GPT":
        result = stream_gpt(prompt,system_message)
    elif model=="Claude":
        result = stream_claude(prompt, system_message)
    elif model == "Gemini":
        result = stream_gemini(prompt, system_message)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result
    

OpenAI API Key existe y empieza por sk-proj-
Anthropic API Key existe y empieza por sk-ant-
Google API Key existe y empieza por AIzaSyCI


In [2]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Nombre de la Empresa:"),
        gr.Textbox(label="Landing page, recuerda incluir http:// o https://"),
        gr.Dropdown(["Divertido","Jovial","Serio","Curioso"], label="Selecciona el tono"),
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Selecciona un modelo")],
    outputs=[gr.Markdown(label="Folleto:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
